In [14]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from collections import Counter

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [15]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [16]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_decomposition.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PGNNIV_PATH = os.path.join(ROOT_PATH, r'results/non_linear/model_POD')


# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PGNNIV_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/model_POD


In [17]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_decomposition.pkl


In [18]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

## División de los datos

In [19]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1).to(DEVICE)
y_train = TensOps(torch.Tensor(dataset['y_train']).unsqueeze(1).requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train']).unsqueeze(1).requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
f_train = TensOps(torch.tensor(dataset['f_train']).unsqueeze(1).to(torch.float32).requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1).to(DEVICE)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).to(torch.float32).unsqueeze(1).requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

Train dataset length: 8000
Validation dataset length: 2000


## POD

In [20]:
U_train, S_train, Vt_train = torch.linalg.svd(y_train.values.detach().squeeze().to('cpu').view(y_train.values.detach().shape[0], -1).T, full_matrices=False)

error = []
for mode_i in range(len(S_train)):
    error.append(1-(sum(S_train[:mode_i])/sum(S_train)).numpy())
    if mode_i < 20:
        print(mode_i, '-->', error[mode_i])

0 --> 1.0
1 --> 0.0983155369758606
2 --> 0.05151933431625366
3 --> 0.00887155532836914
4 --> 0.005898416042327881
5 --> 0.0036424994468688965
6 --> 0.0016180872917175293
7 --> 0.0011507868766784668
8 --> 0.000786125659942627
9 --> 0.0005230903625488281
10 --> 0.0003058314323425293
11 --> 0.00022345781326293945
12 --> 0.0001596212387084961
13 --> 0.00011056661605834961
14 --> 8.106231689453125e-05
15 --> 5.6684017181396484e-05
16 --> 4.3272972106933594e-05
17 --> 3.129243850708008e-05
18 --> 2.372264862060547e-05
19 --> 1.7762184143066406e-05


In [21]:
num_modes = 14

U_reduced_train = U_train[:, :num_modes]
S_reduced_train = S_train[:num_modes]
Vt_reduced_train = Vt_train[:num_modes, :]

modes_base_train = torch.mm(U_reduced_train, torch.diag(S_reduced_train))
# y_train = Vt_reduced_train.T

In [22]:
U_val, S_val, Vt_val = torch.linalg.svd(y_val.values.squeeze().view(y_val.values.shape[0], -1).T, full_matrices=False)

num_modes = 14

U_reduced_val = U_val[:, :num_modes]
S_reduced_val = S_val[:num_modes]
Vt_reduced_val = Vt_val[:num_modes, :]

# data_reconstructed = torch.mm(torch.mm(U_reduced_val, torch.diag(S_reduced_val)), Vt_reduced_val)

# y_val = Vt_reduced_val.T

## Red neuronal general

In [23]:
input_shape = X_train[0].shape
POD_shape = num_modes
output_shape = y_train.values[0].shape

In [24]:
import torch
import torch.nn as nn

from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My

class POD_PGNNIV(nn.Module):
    def __init__(self, input_size, POD_output, explanatory_output_size, POD_base, device, **kwargs):
        super(POD_PGNNIV, self).__init__()

        self.input = input_size
        self.POD_output = POD_output
        self.output_expl = explanatory_output_size

        self.hidden_units_pred = 10
        self.hidden_units_exp = 15
        self.filters_exp = 10

        self.base = POD_base.to(device)

        self.device = device

        # # Predictive network
        self.flatten_layer_pred = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden1_layer_pred = nn.Linear(torch.prod(torch.tensor(self.input, device=self.device)), self.hidden_units_pred).to(self.device)
        self.hidden2_layer_pred = nn.Linear(self.hidden_units_pred, self.hidden_units_pred).to(self.device)
        self.output_layer_pred = nn.Linear(self.hidden_units_pred, self.POD_output).to(self.device)

        # Explanatory network (commented out since they are not used in forward method)
        self.conv1_exp = nn.Conv2d(in_channels=1, out_channels=self.filters_exp, kernel_size=1).to(self.device)
        self.flatten_layer_exp = nn.Flatten().to(self.device)
        self.hidden1_layer_exp = nn.LazyLinear(self.hidden_units_exp).to(self.device)
        self.hidden2_layer_exp = nn.Linear(self.hidden_units_exp, self.hidden_units_exp).to(self.device)
        self.output_layer_exp = nn.Linear(self.hidden_units_exp, self.filters_exp * (self.output_expl[1] - 1) * (self.output_expl[2] - 1)).to(self.device)
        self.conv2_exp = nn.Conv2d(in_channels=self.filters_exp, out_channels=1, kernel_size=1).to(self.device)

    def forward(self, X):

        X = X.to(self.device)

        # Predictive network
        X = self.flatten_layer_pred(X)
        X = torch.sigmoid(self.hidden1_layer_pred(X))
        X = torch.sigmoid(self.hidden2_layer_pred(X))
        output_predictive_net = self.output_layer_pred(X)

        u_pred = torch.mm(self.base, output_predictive_net.T).T.reshape(output_predictive_net.shape[0], self.output_expl[0], self.output_expl[1], self.output_expl[2])
        um_pred = My(Mx(TensOps(u_pred, space_dimension=2, contravariance=0, covariance=0))).values

        x = torch.sigmoid(self.conv1_exp(um_pred))
        x = self.flatten_layer_exp(x)
        x = torch.sigmoid(self.hidden1_layer_exp(x))
        x = torch.sigmoid(self.hidden2_layer_exp(x))
        x = self.output_layer_exp(x)
        x = x.view(x.size(0), self.filters_exp, self.output_expl[1] - 1, self.output_expl[2] - 1)
        K_pred = self.conv2_exp(x)

        return u_pred, K_pred

In [25]:
# Se carga el modelo y el optimizador
POD_model = POD_PGNNIV(input_size=input_shape, POD_output=POD_shape, explanatory_output_size=output_shape, POD_base=modes_base_train, device=DEVICE)
optimizer = torch.optim.Adam(POD_model.parameters(), lr=1e-4)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(POD_model, optimizer, n_checkpoints,
           X_train.to(DEVICE), y_train, X_val, y_val, f_train, f_val,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE,
        )

Start training
Epoch 0, Train loss: 3.526e+11, Test loss: 3.499e+11, MSE(e): 3.525e+04, MSE(pi1): 2.518e+03, MSE(pi2): 1.417e+04, MSE(pi3): 9.845e+01
Epoch 100, Train loss: 4.837e+06, Test loss: 4.577e+06, MSE(e): 4.815e-01, MSE(pi1): 6.363e-01, MSE(pi2): 3.477e-01, MSE(pi3): 1.486e-01
Epoch 200, Train loss: 1.185e+05, Test loss: 1.338e+05, MSE(e): 1.161e-02, MSE(pi1): 8.716e-02, MSE(pi2): 8.434e-03, MSE(pi3): 1.472e-02
Epoch 300, Train loss: 6.678e+04, Test loss: 7.549e+04, MSE(e): 6.580e-03, MSE(pi1): 3.730e-02, MSE(pi2): 5.293e-03, MSE(pi3): 5.939e-03
Epoch 400, Train loss: 5.116e+04, Test loss: 4.584e+04, MSE(e): 5.087e-03, MSE(pi1): 8.889e-03, MSE(pi2): 4.211e-03, MSE(pi3): 1.967e-03
Epoch 500, Train loss: 4.275e+04, Test loss: 3.957e+04, MSE(e): 4.250e-03, MSE(pi1): 9.456e-03, MSE(pi2): 3.668e-03, MSE(pi3): 1.559e-03
Epoch 600, Train loss: 4.388e+04, Test loss: 3.322e+04, MSE(e): 4.364e-03, MSE(pi1): 9.133e-03, MSE(pi2): 3.569e-03, MSE(pi3): 1.442e-03
Epoch 700, Train loss: 3.872